# Дипломный проект - подготовка к созданию прототипа


## Прогнозирование стоимости квартир на побережье Черного моря

Описание:
Нам поставлена задача создать модель, которая будет предсказывать стоимость квартир на Черноморском побережье.
Если наша модель работает хорошо, то мы сможем быстро выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

Датасет был получен при помощи парсинга сайта cian.ru и обработан на этапе EDA.

В данном ноутбуке мы сделаем следующее:
* Обучим модель при помощи алгоритма Random Forest т.к. он работает быстро и достаточно хорошо
* Сохраним модель и названия колонок в отдельных файлах
* Напишем функцию для предсказания стоимости квартиры по входным параметрам
* Сохраним наиболее привлекательные варианты в json-файл

# Загрузка библиотек

In [125]:
import random
import numpy as np 
import pandas as pd 
import sys

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [126]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
Numpy        : 1.18.5


In [127]:
!pip freeze > requirements.txt

# Настройки

In [128]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

TEST_SIZE = 0.2

# Вспомогательные функции

In [129]:
# напишем функцию для расчета mape
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Загружаем данные

In [130]:
df = pd.read_csv('cian_eda.csv')

In [131]:
df.head()

,newBuilding,flatType,floorNumber,fromDeveloper,fullUrl,isApartments,isAuction,kitchenArea,livingArea,roomsCount,...,cargoLiftsCount,materialType,hasBalcony,isBasement,isFirstFloor,isLastFloor,cityPopulation,cityArea,populationDensity,areaPerRoom
0,0,rooms,2,False,https://krym.cian.ru/sale/flat/246745649/,False,True,10.413152,26.000000,1,...,0,monolithBrick,1,0,0,0,79056,18.2,4343.736264,54.40
1,1,rooms,7,True,https://krym.cian.ru/sale/flat/245655396/,False,True,8.093163,24.327497,1,...,0,monolith,0,0,0,0,79056,18.2,4343.736264,42.28
2,1,rooms,6,True,https://krym.cian.ru/sale/flat/247658165/,True,True,10.126025,30.438141,1,...,0,monolith,0,0,0,0,79056,18.2,4343.736264,52.90
3,0,rooms,2,False,https://krym.cian.ru/sale/flat/240865470/,True,True,10.000000,13.809365,1,...,0,monolith,1,0,0,0,79056,18.2,4343.736264,24.00
4,0,rooms,3,False,https://krym.cian.ru/sale/flat/236164313/,False,True,8.613821,25.892559,1,...,1,monolith,1,0,0,0,79056,18.2,4343.736264,45.00


# Предобработка данных

In [132]:
# Составим список бинарных категориальных признаков:
bin_features = [
    'newBuilding',
    'fromDeveloper',
    'isApartments',
    'isAuction',
    'isComplete',
    'hasBalcony',
    'isBasement',
    'isFirstFloor',
    'isLastFloor',
    'passengerLiftsCount',
    'cargoLiftsCount',
]

# Составим список категориальных признаков:
cat_features = [
    'flatType',
    'region',
    'city',
    'materialType',
]
 
# Составим список числовых признаков:
num_features = [
    'floorNumber',
    'kitchenArea',
    'livingArea',
    'roomsCount',
    'totalArea',
    'floorsCount',
    'cityPopulation',
    'cityArea',
    'populationDensity',
    'areaPerRoom',
]

In [133]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df.copy()
    df_output = df_output.drop(['fullUrl'], axis=1)
 
    # # Label Encoding
    for column in bin_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # # One-Hot Encoding:
    df_output = pd.get_dummies(df_output, columns=cat_features, dummy_na=False)
    
    return df_output

In [134]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(df)
df_preproc.sample(10)

,newBuilding,floorNumber,fromDeveloper,isApartments,isAuction,kitchenArea,livingArea,roomsCount,totalArea,price,...,materialType_foamConcreteBlock,materialType_gasSilicateBlock,materialType_monolith,materialType_monolithBrick,materialType_old,materialType_panel,materialType_stalin,materialType_unknown,materialType_wireframe,materialType_wood
27533,0,1,0,0,0,9.100000,40.100000,3,77.0,6499000,...,0,0,0,0,0,0,0,0,0,0
22041,0,1,0,0,0,10.000000,42.000000,2,62.0,3450000,...,0,0,0,0,0,0,0,1,0,0
16454,0,3,0,0,0,8.307551,24.971934,3,43.4,5388000,...,0,0,0,1,0,0,0,0,0,0
22550,0,2,0,0,0,11.867931,35.674192,2,62.0,5500000,...,0,0,0,0,0,0,0,0,0,0
31138,0,8,0,0,0,12.000000,41.000000,3,76.0,6200000,...,0,0,1,0,0,0,0,0,0,0
26754,0,1,0,0,0,17.000000,72.500000,4,109.5,12350000,...,0,0,0,0,0,0,0,1,0,0
9542,1,3,0,0,0,12.200000,50.116486,2,87.1,6454110,...,0,0,1,0,0,0,0,0,0,0
18164,1,5,1,0,0,15.300000,42.800000,3,85.7,4824910,...,0,0,1,0,0,0,0,0,0,0
24138,0,15,0,0,0,11.000000,16.000000,1,38.0,3800000,...,0,0,0,0,0,0,0,1,0,0
19739,0,4,0,0,0,8.600000,41.400000,3,69.4,7000000,...,0,0,0,0,0,0,0,0,0,0


In [135]:
df_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31511 entries, 0 to 31510
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   newBuilding                     31511 non-null  int8   
 1   floorNumber                     31511 non-null  int64  
 2   fromDeveloper                   31511 non-null  int8   
 3   isApartments                    31511 non-null  int8   
 4   isAuction                       31511 non-null  int8   
 5   kitchenArea                     31511 non-null  float64
 6   livingArea                      31511 non-null  float64
 7   roomsCount                      31511 non-null  int64  
 8   totalArea                       31511 non-null  float64
 9   price                           31511 non-null  int64  
 10  floorsCount                     31511 non-null  int64  
 11  isComplete                      31511 non-null  int8   
 12  passengerLiftsCount             

## Split Data

In [136]:
y = df_preproc.price.values
X = df_preproc.drop(['price'], axis=1)

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Обучение модели

## Model 1: RandomForestRegressor

In [138]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [139]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   13.8s finished


RandomForestRegressor(n_jobs=-1, random_state=42, verbose=1)

In [140]:
test_predict_RFR = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_RFR))*100:0.2f}%")

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s


TEST mape: 15.78%


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


Случайный лес работает очень быстро и показывает приемлемые результаты.

# Подготовка к созданию прототипа

In [141]:
# создаем необходимые словари
dic_flatType = {
    'Свободная планировка': 'flatType_openPlan',
    'Комнаты': 'flatType_rooms',
    'Студия': 'flatType_studio'
}
dic_region = {
    'Крым': 'region_crimea',
    'Краснодарский': 'region_krasnodar',
    'Севастополь': 'region_sevastopol'
}
dic_city = {
    'Алушта': 'city_alushta',
    'Анапа': 'city_anapa',
    'Балаклавский район': 'city_balaklavskiy',
    'Евпатория': 'city_evpatoriya',
    'Феодосия': 'city_feodosiya',
    'Гагаринский район': 'city_gagarinskiy',
    'Генелджик': 'city_gelendzhik',
    'Керчь': 'city_kerch',
    'Ленинский район': 'city_leninskiy',
    'Нахимовский район': 'city_nakhimovskiy',
    'Новороссийск': 'city_novorossiysk',
    'Саки': 'city_saki',
    'Сакский район': 'city_sakskiy',
    'Симферополь': 'city_simferopol',
    'Симферопольский район': 'city_simferopolskiy',
    'Сочи': 'city_sochi',
    'Судак': 'city_sudak',
    'Туапсе': 'city_tuapse',
    'Туапсинский район': 'city_tuapsinskiy',
    'Ялта': 'city_yalta',
}
dic_cityPopulation = {
    'Новороссийск': 274956,
    'Анапа': 88879,
    'Сочи': 443562,
    'Симферополь': 342054,
    'Генелджик': 76783,
    'Ялта': 79056,
    'Гагаринский район': 159017,
    'Ленинский район': 58441,
    'Евпатория': 108248,
    'Феодосия': 68001,
    'Алушта': 30088,
    'Керчь': 151548,
    'Нахимовский район': 119507,
    'Симферопольский район': 161997,
    'Туапсе': 61180,
    'Саки': 24654,
    'Балаклавский район': 51092,
    'Сакский район': 76426,
    'Туапсинский район': 127717,
    'Судак': 16489,
}
dic_cityArea = {
    'Новороссийск': 83.5,
    'Анапа': 59,
    'Сочи': 176.8,
    'Симферополь': 107.4,
    'Генелджик': 19.25,
    'Ялта': 18.2,
    'Гагаринский район': 61.1,
    'Ленинский район': 2918.6,
    'Евпатория': 65,
    'Феодосия': 40,
    'Алушта': 7,
    'Керчь': 108,
    'Нахимовский район': 267.7,
    'Симферопольский район': 1752.5,
    'Туапсе': 33.4,
    'Саки': 28.7,
    'Балаклавский район': 530.3,
    'Сакский район': 2257.5,
    'Туапсинский район': 2399.2,
    'Судак': 23.5,
}
dic_materialType = {
    'Газобетонный блок': 'materialType_aerocreteBlock',
    'Блок': 'materialType_block',
    'Кирпич район': 'materialType_brick',
    'Пенобетонный блок': 'materialType_foamConcreteBlock',
    'Газовый силикатный блок': 'materialType_gasSilicateBlock',
    'Монолит': 'materialType_monolith',
    'Монолитный кирпич': 'materialType_monolithBrick',
    'Старый': 'materialType_old',
    'Панель': 'materialType_panel',
    'Сталинка': 'materialType_stalin',
    'Неизвестно': 'materialType_unknown',
    'Каркас': 'materialType_wireframe',
    'Дерево': 'materialType_wood',
}

dic_populationDencity = {k: dic_cityPopulation.get(k, 0) / dic_cityArea.get(k, 0) for k in set(dic_cityPopulation) & set(dic_cityArea)}

Напишем функцию для предсказания цены по параметрам, которую в дальнейшем перенесем на сервер flask с небольшими изменениями

In [142]:
def predict_price(
    newBuilding,
    floorNumber,
    fromDeveloper,
    isApartments,
    isAuction,
    kitchenArea,
    livingArea,
    roomsCount,
    totalArea,
    floorsCount,
    isComplete,
    passengerLiftsCount,
    cargoLiftsCount,
    hasBalcony,
    isBasement,
    isFirstFloor,
    isLastFloor,
    flatType,
    region,
    city,
    materialType    
):
        
    # определяем номер столбца по значению поля
    flatType_index = np.where(X.columns==dic_flatType.get(flatType))[0][0]
    region_index = np.where(X.columns==dic_region.get(region))[0][0]
    city_index = np.where(X.columns==dic_city.get(city))[0][0]
    materialType_index = np.where(X.columns==dic_materialType.get(materialType))[0][0]

    # вычисляем значение признаков
    cityPopulation = dic_cityPopulation.get(city)
    cityArea = dic_cityArea.get(city)
    populationDensity = dic_cityPopulation.get(city) / dic_cityArea.get(city)
    areaPerRoom = totalArea / roomsCount
    
    # заполняем значения переменных
    x = np.zeros(len(X.columns))
    x[0] = newBuilding
    x[1] = floorNumber
    x[2] = fromDeveloper
    x[3] = isApartments
    x[4] = isAuction
    x[5] = kitchenArea
    x[6] = livingArea
    x[7] = roomsCount
    x[8] = totalArea
    x[9] = floorsCount
    x[10] = isComplete
    x[11] = passengerLiftsCount
    x[12] = cargoLiftsCount
    x[13] = hasBalcony
    x[14] = isBasement
    x[15] = isFirstFloor
    x[16] = isLastFloor
    x[17] = cityPopulation
    x[18] = cityArea
    x[19] = populationDensity
    x[20] = areaPerRoom
    
    # заполняем dummy-переменные
    if flatType_index >= 0:
        x[flatType_index] = 1
    if region_index >= 0:
        x[region_index] = 1
    if city_index >= 0:
        x[city_index] = 1
    if materialType_index >= 0:
        x[materialType_index] = 1
    print(x)    
    return model.predict([x])[0]

Убедимся в корректной работе функции

In [143]:
predict_price(0,12,1,0,0,300,300,300,300,18,1,1,1,1,0,0,0,'Комнаты','Крым','Алушта','Монолит')

[0.00000000e+00 1.20000000e+01 1.00000000e+00 0.00000000e+00
 0.00000000e+00 3.00000000e+02 3.00000000e+02 3.00000000e+02
 3.00000000e+02 1.80000000e+01 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.00880000e+04 7.00000000e+00 4.29828571e+03
 1.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


7040700.015

In [144]:
# сохраним модель
import pickle
with open('cian_model_rfr.pickle','wb') as f:
    pickle.dump(model,f)

In [145]:
# сохраним названия колонок
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

Создадим небольшой датасет из 5 наиболее привлекательных объявлений по значению percentage error

In [146]:
# предскажем стоимость для всех квартир
predict_RFR = model.predict(X)
# создадим дополнительные колонки:
# предсказанная цена
df['price_predicted'] = predict_RFR
# процент ошибки - для сортировки по этому параметру
df['error_percent'] = (df.price - df['price_predicted'])/df.price
# величину ошибки в рублях
df['error'] = df['price_predicted'] - df.price
# создадим датасет c сортирофкой по процентной доле ошибки от реальной цены
df_cheap = df[['price','price_predicted','error_percent','error','fullUrl']].sort_values(by=['error_percent']).head(5)
# удалим ненужный параметр
df_cheap = df_cheap.drop(['error_percent'], axis=1).head(5)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


In [147]:
df_cheap

,price,price_predicted,error,fullUrl
17782,1150000,5.847642e+06,4.697642e+06,https://sochi.cian.ru/sale/flat/164596490/
6341,1350000,5.161342e+06,3.811342e+06,https://anapa.cian.ru/sale/flat/246860013/
29219,900000,2.859420e+06,1.959420e+06,https://anapa.cian.ru/sale/flat/240558451/
13505,1950000,6.089016e+06,4.139016e+06,https://sochi.cian.ru/sale/flat/232014644/
13954,2100000,6.460949e+06,4.360949e+06,https://krym.cian.ru/sale/flat/243612621/


Сохраним наиболее выгодные предложения для дальнейшего использования в прототипе

In [148]:
cheap_flats = df_cheap.to_json(orient="index")
with open("cheap_flats.json","w") as f:
    f.write(json.dumps(cheap_flats))